<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Reinforcement Learning for Finance

**Chapter 09 &mdash; Optimal Execution**

&copy; Dr. Yves J. Hilpisch

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

### Please use the "Python 3.10, Tensorflow 2.10" kernel.

## Model Implementation

In [ ]:
!git clone https://github.com/tpq-classes/rl_4_finance.git
import sys
sys.path.append('rl_4_finance')


In [ ]:
import math
import random
import numpy as np
import pandas as pd
from pylab import plt, mpl

In [ ]:
plt.style.use('seaborn-v0_8')
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True)

In [ ]:
class AlmgrenChriss:
    def __init__(self, T, N, S0, sigma, X, gamma, eta, lamb):
        self.T = T              
        self.N = N           
        self.dt = T / N
        self.S0 = S0
        self.sigma = sigma
        self.X = X
        self.gamma = gamma
        self.eta = eta
        self.lamb = lamb

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def optimal_execution(self):
        kappa = np.sqrt(self.lamb * self.sigma ** 2 / self.eta)
        t = np.linspace(0, self.T, self.N + 1)
        xt_sum = (self.X * np.sinh(kappa * (self.T - t)) /
                  np.sinh(kappa * self.T))
        xt = -np.diff(xt_sum, prepend=0)
        xt[0] = 0
        return t, xt

In [ ]:
T = 10  # <1>
N = 10  # <2>
S0 = 1  # <3>
sigma = 0.15  # <4>
X = 1  # <5>
gamma = 0.1  # <6>
eta = 0.1  # <7>
lamb_high = 0.2  # <8>
lamb_low = 0.0001  # <8>

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, gamma, eta, lamb_high)

In [ ]:
t, xth = ac.optimal_execution()

In [ ]:
t

In [ ]:
xth.round(3)  # <9>

In [ ]:
ac.lamb = lamb_low

In [ ]:
t, xtl = ac.optimal_execution()
xtl.round(3)  # <10>

In [ ]:
plt.plot(t, ac.X - xth.cumsum(), 'r', lw=1,
         label='high $\\lambda$ (position)')
plt.plot(t, xth, 'rs', markersize=4,
         label='high $\\lambda$ (trade)')
plt.plot(t, ac.X- xtl.cumsum(), 'b--', lw=1,
         label='low $\\lambda$ (position)')
plt.plot(t, xtl, 'bo', markersize=4,
         label='low $\\lambda$ (trade)')
plt.xlabel('trading day')
plt.ylabel('shares (normalized to 1)')
plt.legend();

In [ ]:
from numpy.random import default_rng

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def simulate_stock_price(self, xt, seed=None):
        rng = default_rng(seed=seed)
        S = np.zeros(self.N + 1)  # <1>
        S[0] = self.S0  # <1>
        P = np.zeros(self.N + 1)  # <2>
        P[0] = self.S0  # <2>
        for t in range(1, self.N + 1):
            dZ = rng.normal(0, np.sqrt(self.dt))
            S[t] = S[t - 1] + sigma * dZ  # <1>
            P[t] = S[t] - self.gamma * xt[:t + 1].sum()  # <2>
        return S, P

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, gamma, eta, lamb_high)

In [ ]:
t, xth = ac.optimal_execution()

In [ ]:
xth.round(3)

In [ ]:
seed = 250

In [ ]:
S, Ph = ac.simulate_stock_price(xth, seed=seed)

In [ ]:
ac.lamb = lamb_low

In [ ]:
t, xtl = ac.optimal_execution()

In [ ]:
xtl.round(2)

In [ ]:
S, Pl = ac.simulate_stock_price(xtl, seed=seed)

In [ ]:
plt.plot(t, S, 'b', lw=1, label='simulated stock price path')
plt.plot(t, Ph, 'r--', lw=1, label='adjusted path (high $\\lambda$)')
plt.plot(t, Pl, 'g:', lw=1, label='adjusted path (low $\\lambda$)')
plt.xlabel('trading day')
plt.ylabel('stock price (normalized to 1)')
plt.legend();

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def calculate_costs(self, xt):
        temporary_cost = np.sum(self.eta *
                    (xt / self.dt) ** 2 * self.dt)
        permanent_cost = np.sum(self.gamma * np.cumsum(xt) * xt)
        execution_risk = self.lamb * self.sigma ** 2 * np.sum(
            (np.cumsum(xt[::-1])[::-1] / self.dt) ** 2 * self.dt)
        TEC = temporary_cost + permanent_cost + execution_risk
        return temporary_cost, permanent_cost, execution_risk, TEC

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, gamma, eta, lamb_high)

In [ ]:
t, xth = ac.optimal_execution()

In [ ]:
tc, pc, er, TEC = ac.calculate_costs(xth)

In [ ]:
print(f'lambda = {ac.lamb}')
print(f'temporary cost = {tc:7.4f}')
print(f'permanent cost = {pc:7.4f}')
print(f'execution risk = {er:7.4f}')
print(f'total ex. cost = {TEC:7.4f}')  # <1>

In [ ]:
ac.lamb = lamb_low

In [ ]:
t, xtl = ac.optimal_execution()

In [ ]:
tc, pc, er, TEC = ac.calculate_costs(xtl)

In [ ]:
print(f'lambda = {ac.lamb}')
print(f'temporary cost = {tc:7.4f}')
print(f'permanent cost = {pc:7.4f}')
print(f'execution risk = {er:7.4f}')
print(f'total ex. cost = {TEC:7.4f}')  # <2>

## Execution Environment

In [ ]:
class Execution:
    def __init__(self, T, N, sigma, X, gamma, eta, lamb):
        self.T = T              
        self.N = N           
        self.dt = T / N
        self.sigma = sigma
        self.X = X
        self.gamma = gamma
        self.eta = eta
        self.lamb = lamb
        self.episode = 0

In [ ]:
class Execution(Execution):
    def _get_state(self):
        s = np.array([self.X_,  # <1>
                    self.bar / self.N])  # <2>
        state = np.hstack((self.xt, s))  # <3>
        return state, {}
    def reset(self):
        self.bar = 0
        self.treward = 0
        self.episode += 1
        self.X_ = self.X  # <1>
        self.xt = np.zeros(self.N + 1)  # <4>
        self.tec = pd.DataFrame(
            {'pc': 0, 'tc': 0, 'er': 0}, index=[0])  # <5>
        return self._get_state()

In [ ]:
class Execution(Execution):
    def step(self, action):
        self.bar += 1
        self.xt[self.bar] = action  # <1>
        self.X_ -= action  # <2>
        pc = np.sum(self.gamma *
                np.cumsum(self.xt) * self.xt)  # <3>
        tc = np.sum(self.eta *
                (self.xt / self.dt) ** 2 * self.dt)  # <3>
        er = self.lamb * self.sigma ** 2 * np.sum(
            (np.cumsum(self.xt[::-1])[::-1] / self.dt) ** 2
            * self.dt)  # <3>
        df = pd.DataFrame({'pc': tc, 'tc': pc, 'er': er},
                          index=[0])  # <3>
        self.tec = pd.concat((self.tec, df))  # <3>
        cost = self.tec.diff().fillna(0).iloc[-1]  # <3>
        tec = cost.sum()  # <3>
        self.state, _ = self._get_state()
        pen = 0
        if self.bar < self.N:
            if self.X_ <= 0.0001:
                done = True
            else:
                done = False
        elif self.bar == self.N:
            pen = abs(self.X_) * 10  # <4>
            done = True
        return self.state, -(tec + pen), done, False, {}

In [ ]:
execution = Execution(T, N, sigma, X, gamma, eta, lamb_low)

In [ ]:
execution.reset()
execution.step(1.0)  # <1>

In [ ]:
execution.reset()

In [ ]:
execution.step(0.5)  # <2>

In [ ]:
execution.step(0.5)  # <3>

In [ ]:
execution.reset()
cost = list()
for i in range(10):
    cost.append(execution.step(0.1)[1])  # <4>
print(f'TEC = {sum(cost):.3f}')

## Random Agent

In [ ]:
execution = Execution(T, N, sigma, X, gamma, eta, lamb_low)  # <1>

In [ ]:
rng = default_rng(seed=100)

In [ ]:
def gen_rn():
    alpha = np.ones(N)  # <2>
    rn = rng.dirichlet(alpha)  # <2>
    rn = np.insert(rn, 0, 0)  # <3>
    return rn

In [ ]:
rn = gen_rn()
rn  # <4>

In [ ]:
rn.sum()  # <5>

In [ ]:
def execute_trades():
    for _ in range(5):
        execution.reset()
        rn = gen_rn()
        for i in range(1, 11):
            execution.step(rn[i])  # <6>
        tec = execution.tec.iloc[-1].sum()
        print(f'TEC = {tec:.3f}')

In [ ]:
execute_trades()  # <7>

In [ ]:
execution = Execution(T, N, sigma, X, gamma, eta, lamb_high)  # <8>

In [ ]:
execute_trades()  # <9>

## Execution Agent

In [ ]:
import os
import warnings
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from dqlagent import *

In [ ]:
random.seed(100)
tf.random.set_seed(100)

In [ ]:
opt = keras.optimizers.legacy.Adam

In [ ]:
class ExecutionAgent(DQLAgent):
    def __init__(self, symbol, feature, n_features, env,
                 hu=24, lr=0.0001, rng='equal'):
        self.epsilon = 1.0
        self.epsilon_decay = 0.9975
        self.epsilon_min = 0.1
        self.memory = deque(maxlen=2000)
        self.batch_size = 32
        self.eta = 1.0
        self.trewards = list()
        self.max_treward = -np.inf
        self.n_features = n_features
        self.env = env
        self.episodes = 0
        self.rng = rng
        self._generate_rn()  # <1>
        self.actor = self._create_model(hu, lr, 'sigmoid')  # <2>
        self.critic = self._create_model(hu, lr, 'linear')  # <3>

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def _generate_rn(self):
        if self.rng == 'equal':
            alpha = np.ones(self.env.N)  # <1>
        elif self.rng == 'decreasing':
            alpha = range(self.env.N, 0, -1)   # <2>
        else:
            alpha = rng.random(self.env.N)  # <3>
        rn = rng.dirichlet(alpha)
        self.rn = np.insert(rn, 0, 0)

In [ ]:
class ExecutionAgent(ExecutionAgent): 
    def _create_model(self, hu, lr, out_activation):
        model = Sequential()
        model.add(Dense(hu, input_dim=self.n_features,
                        activation='relu'))
        model.add(Dense(hu, activation='relu'))
        model.add(Dense(1, activation=out_activation))
        model.compile(loss='mse', optimizer=opt(learning_rate=lr))
        return model

In [ ]:
class ExecutionAgent(ExecutionAgent):    
    def act(self, state):
        if random.random() <= self.epsilon or self.episodes < 250:  # <1>
            return min(self.rn[self.f], state[0, -2])  # <2>
        else:
            action = self.actor(tf.convert_to_tensor(state, dtype=tf.float32), training=False).numpy()[0, 0]  # <3>
        return action

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
        for state, action, next_state, reward, done in batch:
            target = reward
            if not done:
                target += self.eta * self.critic(tf.convert_to_tensor(next_state, dtype=tf.float32), training=False).numpy()[0, 0]  # <1>
                self.critic.fit(state, np.array([target]),
                        epochs=1, verbose=False)  # <2>
                # advantage = target - self.critic(tf.convert_to_tensor(state, dtype=tf.float32), training=False).numpy()[0, 0]
                self.actor.fit(state, np.array([action]),
                        # sample_weight=np.array([advantage]),
                        epochs=1, verbose=False)  # <3> 
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        self._generate_rn()  # <4>

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def test(self, episodes, verbose=True):
        for e in range(1, episodes + 1):
            state, _ = self.env.reset()
            state = self._reshape(state)
            treward = 0
            for _ in range(1, self.env.N + 1):
                action = self.actor(tf.convert_to_tensor(state, dtype=tf.float32), training=False).numpy()[0, 0]  # <1>
                state, reward, done, trunc, _ = self.env.step(action)
                state = self._reshape(state)
                treward += reward
                if done:
                    templ = f'total reward={treward:4.3f}'
                    if verbose:
                        print(templ)
                    break
            print(self.env.xt)

In [ ]:
execution = Execution(T, N, sigma, X, gamma, eta, lamb_low)

In [ ]:
executionagent = ExecutionAgent(None, feature=None,
                    n_features=execution.N + 3,
                    env=execution, hu=64, lr=0.0001,
                    rng='equal')

In [ ]:
episodes = 2500

In [ ]:
%time executionagent.learn(episodes)

In [ ]:
executionagent.test(1)

In [ ]:
xtl_ = execution.xt
xtl_.sum()

In [ ]:
execution = Execution(T, N, sigma, X, gamma, eta, lamb_high)

In [ ]:
executionagent = ExecutionAgent(None, feature=None,
                    n_features=execution.N + 3,
                    env=execution, hu=64, lr=0.0001,
                    rng='decreasing')

In [ ]:
%time executionagent.learn(episodes)

In [ ]:
executionagent.test(1)

In [ ]:
xth_ = execution.xt
xth_.sum()

In [ ]:
plt.plot(xtl[1:], 'b', lw=1, label='optimal for low $\lambda$')
plt.plot(xtl_[1:], 'b:', lw=1, label='learned for low $\lambda$')
plt.plot(xth[1:], 'r--', lw=1, label='optimal for high $\lambda$')
plt.plot(xth_[1:], 'r-.', lw=1, label='learned for high $\lambda$')
plt.xlabel('trading day')
plt.ylabel('trade size')
plt.legend();

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>